# M5 Agentic AI - 客服代理

## 1. 引言

正如 Andrew 在讲座中解释的那样，*结合代码执行的规划*意味着让 LLM **编写代码并将其作为计划本身**。  
与纯文本或基于 JSON 的计划相比，这种方法更具表达力和灵活性：代码不仅记录步骤，还能直接执行它们。

在本实验中，你将实践这一设计模式。  
我们不再让 LLM 输出 JSON 计划并手动执行每一步，而是允许它**编写 Python 代码**来直接表达一个计划的多步逻辑。通过执行这些代码，我们可以自动完成复杂查询。  

为更具体地展示这一点，我们模拟一个**太阳镜商店**，包含产品**库存**与一组**交易**（销售、退货、余额更新）。该示例展示了 LLM 如何生成用于查询或更新记录的代码，体现这一模式的灵活性。

### 1.1 实验概览
我们将：
1. 创建简单的**库存**与**交易**数据集。  
2. 构建描述数据的**模式块（schema block）**。  
3. 通过提示让 LLM **用 Python 代码编写计划**（并在代码注释中解释每一步）。  
4. 在沙箱中执行代码以获得答案。  

### 1.2 学习目标

完成本实验后，你将能够：

- **说明**为何让模型编写代码（而非 JSON 或纯文本计划）能实现更丰富、更灵活的规划。  
- **提示** LLM 生成带有逐步注释的 Python 代码，使其既能文档化也能执行计划。  
- **安全运行**生成的代码于沙箱中，并解释结果。  

这说明了为何*以代码为行动（Code as Action）*能够优于脆弱的工具链与基于 JSON 的规划方式。

## 2. 环境设置

In [ ]:
# ==== 导入 ====
from __future__ import annotations
import json
from dotenv import load_dotenv
from openai import OpenAI
import re, io, sys, traceback, json
from typing import Any, Dict, Optional
from tinydb import Query, where

# 工具模块
import utils      # 用于提示/打印的助手函数
import inv_utils  # 用于库存、交易、模式构建和 TinyDB 数据填充的函数

load_dotenv()
client = OpenAI()

在 `inv_utils` 模块中，包含如下函数：

- `create_inventory()` – 构建太阳镜库存。  
- `create_transactions()` – 构建初始交易日志。  
- `seed_db()` – 将库存与交易加载到基于 JSON 的存储中。  
- `build_schema_block()` – 生成在提示中使用的模式描述。  
- 还有诸如 `get_current_balance()` 与 `next_transaction_id()` 的辅助函数——让 LLM 在库存与交易之间执行一致的更新。  

### 2.1 创建示例表

我们将使用 **[TinyDB](https://tinydb.readthedocs.io/)** 为太阳镜商店模拟创建两个小型表。TinyDB 是一个用纯 Python 编写的轻量级文档型数据库。  
TinyDB 以 JSON 文档的形式存储数据，适合小型应用或原型，因为它无需服务器配置，并且可轻松查询与更新数据。

两个表为：

- **`inventory_tbl`**：包含产品详情，如名称、商品 ID、描述、库存数量、价格等。  
- **`transactions_tbl`**：以期初余额开头，随后记录购买、退货和调整。  

你将使用 `inv_utils` 中的辅助函数生成这些表，并在下面预览前几行。

In [ ]:
db, inventory_tbl, transactions_tbl = inv_utils.seed_db()

现在，你可以通过打印格式化的 JSON 来检查各表中的记录：

In [ ]:
utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表")
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表")

如上所示，各表的模式如下：

<div style="border:1px solid #BFDBFE; border-left:6px solid #3B82F6; background:#EFF6FF; border-radius:6px; padding:16px; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif; line-height:1.6; color:#1E3A8A;">

  <h4 style="margin-top:0; color:#1E40AF;">库存表（<code>inventory_tbl</code>）</h4>
  <ul>
    <li><strong>item_id</strong>（字符串）：唯一产品标识（例如 SG001）。</li>
    <li><strong>name</strong>（字符串）：太阳镜款式（例如 Aviator、Round）。</li>
    <li><strong>description</strong>（字符串）：产品的文本描述。</li>
    <li><strong>quantity_in_stock</strong>（整数）：当前可用库存。</li>
    <li><strong>price</strong>（浮点）：以美元计价。</li>
  </ul>
  <h4 style="margin-top:1em; color:#1E40AF;">交易表（<code>transactions_tbl</code>）</h4>
  <ul>
    <li><strong>transaction_id</strong>（字符串）：唯一交易标识（例如 TXN001）。</li>
    <li><strong>customer_name</strong>（字符串）：顾客姓名，或期初条目的 <code>OPENING_BALANCE</code>。</li>
    <li><strong>transaction_summary</strong>（字符串）：交易的简短描述。</li>
    <li><strong>transaction_amount</strong>（浮点）：该笔交易的金额。</li>
    <li><strong>balance_after_transaction</strong>（浮点）：应用该交易后的余额。</li>
    <li><strong>timestamp</strong>（字符串）：ISO-8601 格式的交易日期/时间。</li>
  </ul>
</div>


## 结合代码执行的规划

### 2.1. 计划

在明确了模式后，你将构建一个**提示**，使模型通过*编写代码来规划*，并随后执行该代码。正如 Andrew 强调的：代码即计划。模型在注释中解释每一步，然后执行它。下面的提示还让模型自行判断请求是只读还是状态变更，并强制安全执行（无 I/O、无网络、仅 TinyDB 查询、变更一致）。


In [ ]:
PROMPT = """你是一名高级数据助手。通过编写 TINDYDB PYTHON 代码来制定计划。

数据库模式和示例 (只读):
{schema_block}

执行环境 (已导入/已提供):
- 变量: db, inventory_tbl, transactions_tbl  # TinyDB 表对象
- 助手函数: get_current_balance(tbl) -> float, next_transaction_id(tbl, prefix="TXN") -> str
- 自然语言: user_request: str  # 原始用户消息

规划规则 (关键):
- 从 user_request 派生所有过滤器/参数 (形状/关键词, 价格范围 "低于/高于/之间", 库存提及, 数量, 购买/退货意图)。不要硬编码值。
- 使用 Query() 动态构建 TinyDB 查询。如果 user_request 中没有某个约束，则不要应用它。
- 保持保守：如果意图不明确，请执行只读操作 (演习 - DRY RUN)。

交易政策 (硬性规定):
- 不要创建聚合的多项目交易。
- 如果请求包含多个项目，则为每个项目创建单独的交易行。
- 对于每个项目：
  - 计算其自己的单行总计 (unit_price * qty),
  - 插入一笔该金额的交易,
  - 按顺序更新余额 (balance += line_total),
  - 更新该项目的库存。
- 如果任何请求的项目库存不足，不要更改任何数据；回复 STATUS="insufficient_stock"。

人类可读响应要求 (硬性规定):
- 你必须设置一个名为 `answer_text` (str 类型) 的变量，内容为简短、客户友好的句子 (1-2 行)。
- 这个句子是唯一面向用户的消息。没有数据帧/JSON，没有样板式的免责声明。
- 如果没有匹配项，礼貌地说明情况，并提供一个相近的替代方案 (最接近的款式/价格) 或下一步建议。

行动政策:
- 如果请求明确要求改变状态 (购买/采购/退货/补货/调整):
    ACTION="mutate"; SHOULD_MUTATE=True; 执行更改并写入匹配的交易行。
  否则:
    ACTION="read"; SHOULD_MUTATE=False; 模拟并作为演习 (dry run) 简要说明 (仅在日志中)。

失败与边缘情况处理 (必须实现):
- 不要在 Query.test 中捕获外部变量。将它们作为显式参数传递。
- 始终设置一个简短的 `answer_text`。同时设置一个字符串 `STATUS`，其值为以下之一:
  "success", "no_match", "insufficient_stock", "invalid_request", "unsupported_intent"。
- no_match: 没有项目满足过滤器 → 建议风格/价格最接近的，或邀请客户提供不同范围。
- insufficient_stock: 找到项目但库存 < 请求数量 → 说明可用数量，并提供你能满足的最大数量。
- invalid_request: 无法解析基本信息 (例如 购买/退货 的数量) → 简洁地询问缺失的部分。
- unsupported_intent: 该行动超出了商店的能力范围 → 提供最接近的支持替代方案。
- 在所有情况下，保持乐于助人且简洁的语气 (1-2 句话)。仅在 stdout 日志中放入技术细节 (例如 ACTION/DRY RUN)。

输出契约:
- 仅在这些标签之间返回可执行的 Python (没有额外文本):
  <execute_python>
  # 你的 python
  </execute_python>

代码核对清单 (在代码中遵循):
1) 从 user_request 解析意图和约束 (可用正则表达式)。
2) 渐进式构建 TinyDB 条件；查询 inventory_tbl。
3) 如果是变更(mutate)操作: 验证库存, 更新库存, 插入一笔交易 (新 id, 金额, 余额, 时间戳)。
4) 始终设置:
   - `answer_text` (人类可读的句子, 必需),
   - `STATUS` (参见上面的列表)。
   同时向 stdout 打印一个简短的日志, 例如 "LOG: ACTION=read DRY_RUN=True STATUS=no_match"。
5) 可选: 如果有用的话设置 `answer_rows` 或 `answer_json`，但 `answer_text` 是强制性的。

语气示例 (用于 `answer_text`):
- 成功: "是的，我们有经典款太阳镜，这是一款圆形镜框，售价60美元。"
- 无匹配: "我们目前没有100美元以下的圆形镜框库存，但我们的 Moon圆形镜框有货，售价120美元。"
- 库存不足: "Classic款我们只剩下1副了；我可以为您预留。"
- 无效请求: "我可以处理这个——请问您想购买多少副？"
- 不支持的意图: "我们不能翻新镜框，但我可以推荐类似的新款式。"

约束条件:
- 使用 TinyDB Query 进行过滤。仅在需要时使用标准库导入。
- 保持代码清晰，并使用编号步骤进行注释。

用户请求:
{question}
"""

### 2.2 从提示到代码（在代码中规划）

让我们生成一段**即为计划本身**的代码。

与其让模型输出 JSON 计划并用许多小工具逐步执行，不如让它**编写一段涵盖整个计划的 Python 代码**（例如，“先过滤这个，再计算那个，然后更新该行”）。函数 `generate_llm_code`：

1. 从 `inventory_tbl` 与 `transactions_tbl` **构建实时模式**，让模型看到真实字段、类型与示例。
2. 使用该模式和用户问题**格式化提示**。
3. **调用模型**生成**带代码的计划**响应——通常是一个 `<execute_python>...</execute_python>` 代码块，内部包含逐步逻辑。
4. **返回完整响应**（包含计划与代码）。  
   *此步不执行任何代码。*

为何采用此模式？充分利用模型已“熟悉”的 Python/TinyDB 作为丰富工具箱，使其能直接在代码中组合多步解法，而不是依赖不断膨胀的定制工具集。我们将在后续步骤中提取并运行代码。

In [ ]:
# ---------- 1) 代码生成 ----------
def generate_llm_code(
    prompt: str,
    *,
    inventory_tbl,
    transactions_tbl,
    model: str = "gpt-4.1-mini",
    temperature: float = 0.2,
) -> str:
    """
    请求 LLM 生成一个“带代码的计划”响应。
    返回完整的助手内容 (包括周围的文本和标签)。
    实际的代码提取稍后在 execute_generated_code 中进行。
    """
    schema_block = inv_utils.build_schema_block(inventory_tbl, transactions_tbl)
    prompt = PROMPT.format(schema_block=schema_block, question=prompt)

    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": "你编写安全、注释良好的 TinyDB 代码来处理数据问题和更新。"
            },
            {"role": "user", "content": prompt},
        ],
    )
    content = resp.choices[0].message.content or ""
    
    return content

### 2.3 尝试一个示例提示（在代码中规划）

我们使用 Andrew 在讲座中的同一个提示：

> **提示：** “你们是否有库存中的圆形太阳镜，且价格低于 100 美元？”

在生成代码之前，先人工检查 TinyDB 表，确认是否确有*圆形*框（仅按词匹配）以及价格情况。运行下一个单元以预览库存，并突出显示与“round”词匹配的条目。

In [ ]:
Item = Query()                  # 创建一个 Query 对象来引用字段 (例如 Item.name, Item.description)

# 搜索库存表，查找描述(description)或名称(name)
# 包含单词 "round" (不区分大小写) 的文档。检查是内联完成的：
# - (v or "") 确保我们通过将 None 转换为空字符串来处理它
# - .lower() 规范化大小写
# - " round " 强制执行一个粗略的单词边界 (不会匹配 "wraparound")
round_sunglasses = inventory_tbl.search(
    (Item.description.test(lambda v: " round " in ((v or "").lower()))) |
    (Item.name.test(        lambda v: " round " in ((v or "").lower())))
)

# 在 notebook UI 中将结果渲染为格式化的 JSON
utils.print_html(json.dumps(round_sunglasses, indent=2), title="Inventory Status: Round Sunglasses")

好的——我们确实有圆形镜框。根据我们的人工检查，有两款圆形镜框有库存，但只有**一款**价格**低于100美元**。因此，满足该要求的商品是：

```python
{
  "item_id": "SG005",
  "name": "Classic",
  "description": "经典的圆形轮廓，搭配简约的金属镜框，提供永恒而百搭的风格，适合休闲和正式场合穿搭。",
  "quantity_in_stock": 10,
  "price": 60
}
```

现在，让我们要求模型**生成一个代码计划**来回答 Andrew 的提示（先不执行）。

In [ ]:
# 讲座中的 Andrew 示例提示
prompt_round = "你们是否有库存中的圆形太阳镜，且价格低于 100 美元？"

# 生成以代码为计划的响应（完整内容；可能包含 <execute_python> 标签）
full_content_round = generate_llm_code(
    prompt_round,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="o4-mini",
    temperature=1.0,
)

# 查看 LLM 的计划与代码（此处不执行）
utils.print_html(full_content_round, title="以代码为计划（完整响应）")

### 2.4. 定义执行器函数（运行给定计划）

我们将定义一个函数，**安全地接收模型生成的计划并运行**：

- 它**既可接收**完整的 LLM 响应（包含 `<execute_python>…</execute_python>`），**也可接收**原始 Python 代码。
- 需要时**提取**可执行代码块。
- 在**受控命名空间**中运行代码（仅 TinyDB 表与安全辅助函数）。
- 捕获**标准输出**、**错误**以及模型设置的答案变量（`answer_text`、`answer_rows`、`answer_json`）。
- 渲染**前/后**表快照，明确展示副作用。

这是将**以代码为计划**的结果转化为行动与简洁用户答案的“执行器”。


In [ ]:
# --- 辅助函数：提取 <execute_python>...</execute_python> 之间的代码 ---
def _extract_execute_block(text: str) -> str:
    """
    返回 <execute_python>...</execute_python> 标签内的 Python 代码。
    如果未找到标签，则假定 'text' 已经是原始 Python 代码。
    """
    if not text:
        raise RuntimeError("空内容被传递给代码执行器。")
    m = re.search(r"<execute_python>(.*?)</execute_python>", text, re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else text.strip()


# ---------- 2) 代码执行 ----------
def execute_generated_code(
    code_or_content: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    user_request: Optional[str] = None,
) -> Dict[str, Any]:
    """
    在受控的命名空间中执行代码。
    接受原始 Python 代码 或 包含 <execute_python> 标签的完整内容。
    返回最小化的产物：stdout、error 和提取的答案。
    """
    # 在此处提取代码 (现在是集中处理)
    code = _extract_execute_block(code_or_content)

    SAFE_GLOBALS = {
        "Query": Query,
        "get_current_balance": inv_utils.get_current_balance,
        "next_transaction_id": inv_utils.next_transaction_id,
        "user_request": user_request or "",
    }
    SAFE_LOCALS = {
        "db": db,
        "inventory_tbl": inventory_tbl,
        "transactions_tbl": transactions_tbl,
    }

    # 捕获从被执行代码产生的 stdout
    _stdout_buf, _old_stdout = io.StringIO(), sys.stdout
    sys.stdout = _stdout_buf
    err_text = None
    try:
        exec(code, SAFE_GLOBALS, SAFE_LOCALS)
    except Exception:
        err_text = traceback.format_exc()
    finally:
        sys.stdout = _old_stdout
    printed = _stdout_buf.getvalue().strip()

    # 提取由生成代码设置的可能答案
    answer = (
        SAFE_LOCALS.get("answer_text")
        or SAFE_LOCALS.get("answer_rows")
        or SAFE_LOCALS.get("answer_json")
    )


    return {
        "code": code,           # <- 已经没有标签了
        "stdout": printed,
        "error": err_text,
        "answer": answer,
        "transactions_tbl": transactions_tbl.all(), # 供检查
        "inventory_tbl": inventory_tbl.all(),     # 供检查
    }

你已检查库存并确认：低于 100 美元的圆形款式恰有一款。现在到有趣的部分：将模型的以代码为计划交给执行器，看看它如何完成工作。执行器会提取 <code><execute_python>...</execute_python></code> 代码块，在受限沙箱中运行，并展示所有关键信息——表发生了什么变化（前/后）、计划打印的日志，以及最终的、面向用户的简洁 answer_text。

In [ ]:
# 针对圆形太阳镜问题执行生成的计划
result = execute_generated_code(
    full_content_round,          # 先前生成的完整 LLM 响应
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_round,   # 例如："你们是否有库存中的圆形太阳镜，且价格低于 100 美元？"
)

# 查看计划实际执行的 Python 所提取的答案
utils.print_html(result["answer"], title="计划执行 · 提取的答案")

可以看到，这与我们先前的人工分析所得结果一致。

## 2.4 退回两副 Aviator 太阳镜

在先前步骤中我们只是**查询**数据，因此库存与交易未发生变化。  
现在使用在代码中规划的模式来处理一个**退货**场景：
> **请求：** “退回我上周购买的 2 副 Aviator 太阳镜。”

在生成计划之前，先**检查当前库存**中的 *Aviator* 款。

In [ ]:
Item = Query()                    # 创建 Query 对象以引用字段（例如 Item.name、Item.description）

# 查询：提取所有 'name' 恰为 "Aviator" 的库存行。
# 说明：
# - 这是区分大小写的相等判断。"aviator" 不会匹配。
# - 若需不区分大小写匹配，可考虑 .test(...) 或 .matches(..., re.I)。
aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

# 以可读的 JSON 面板展示匹配文档
utils.print_html(json.dumps(aviators, indent=2), title="库存状态：退货前的 Aviator 太阳镜")

库存显示一款 Aviator SKU 在库——**SG001 (Aviator)**：**23** 件，每件 **$80**。现在生成一个计划来回答该请求：

In [ ]:
prompt_aviator = "退回我上周购买的 2 副 Aviator 太阳镜。"

# 生成以代码为计划的响应（完整内容；可能包含 <execute_python> 标签）
full_content_aviator = generate_llm_code(
    prompt_aviator,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="o4-mini",
    temperature=1,
)

# 查看 LLM 的计划与代码（此处不执行）
utils.print_html(full_content_aviator, title="以代码为计划（完整响应）")

在执行计划之前，先检查当前的交易状态。

In [ ]:
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="退货前的交易表")

当前交易日志仅显示一条记录——期初余额（`TXN001`）为 `$500.00`，记录时间 `2025-10-03T09:16:59.628898`。 

准备就绪——运行下方单元以执行计划。

In [ ]:
# 执行针对 Aviator 太阳镜退货的生成计划
result = execute_generated_code(
    full_content_aviator,          # 先前生成的完整 LLM 响应
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_aviator,   # 例如："退回我上周购买的 2 副 Aviator 太阳镜。"
)

# 查看计划实际执行的 Python 所提取的答案
utils.print_html(result["answer"], title="计划执行 · 提取的答案")

如下所示，已为 Aviator 太阳镜退货插入一条新交易记录。

In [ ]:
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="退货后的交易表")

运行下方单元后，你将看到 Aviator 的库存增加到 25（`quantity_in_stock`）。

In [ ]:
Item = Query()                  

aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

utils.print_html(json.dumps(aviators, indent=2), title="库存状态：退货后的 Aviator 太阳镜")

## 3. 综合起来：客服代理

你已构建各个组件——模式、提示、代码生成器与执行器。现在将它们串联为一个助手：接收自然语言请求，生成以代码为计划，安全执行，并展示结果（含前/后表快照）。

**该代理的功能**
- 可选地重置演示数据以获得干净运行。
- 生成计划（Python 位于 `<execute_python>…</execute_python>` 中）。
- 在受控命名空间中执行计划（TinyDB + 辅助函数）。
- 输出简洁的 `answer_text` 并渲染前/后快照。

In [ ]:
def customer_service_agent(
    question: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    model: str = "o4-mini",
    temperature: float = 1.0,
    reseed: bool = False,
) -> dict:
    """
    端到端助手：
      1) (可选) 重新填充库存和交易数据
      2) 根据 `question` 生成“代码即计划”
      3) 在受控的命名空间中执行
      4) 渲染执行前后的快照并返回产物

    Returns:
      {
        "full_content": <原始 LLM 响应 (可能包含 <execute_python> 标签)>,
        "exec": {
            "code": <提取出的 python 代码>,
            "stdout": <计划日志>,
            "error": <回溯信息或 None>,
            "answer": <答案文本/行/json>,
            "inventory_after": [...],
            "transactions_after": [...]
        }
      }
    """
    # 0) 可选的重新填充数据
    if reseed:
        inv_utils.create_inventory()
        inv_utils.create_transactions()

    # 1) 显示问题
    utils.print_html(question, title="用户问题")

    # 2) 生成“代码即计划”(完整内容)
    full_content = generate_llm_code(
        question,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        model=model,
        temperature=temperature,
    )
    utils.print_html(full_content, title="以代码为计划（完整响应）")

    # 3) 执行前的快照
    utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表 · 之前")
    utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表 · 之前")

    # 4) 执行
    exec_res = execute_generated_code(
        full_content,
        db=db,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        user_request=question,
    )

    # 5) 执行后的快照 + 最终答案
    utils.print_html(exec_res["answer"], title="计划执行 · 提取的答案")
    utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表 · 之后")
    utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表 · 之后")

    # 6) 返回产物
    return {
        "full_content": full_content,
        "exec": {
            "code": exec_res["code"],
            "stdout": exec_res["stdout"],
            "error": exec_res["error"],
            "answer": exec_res["answer"],
            "inventory_after": inventory_tbl.all(),
            "transactions_after": transactions_tbl.all(),
        },
    }

## 4. 试用（借助客服代理）

使用 `customer_service_agent(...)` 助手，从自然语言请求 → 以代码为计划 → 安全执行 → 前/后快照。

**试试这些提示：**
1）**只读（Andrew 的示例）：**  
   “你们是否有库存中的圆形太阳镜，且价格低于 100 美元？”
2）**变更——退货：**  
   “退回 2 副 Aviator 太阳镜。”
3）**变更——购买：**  
   “为顾客 Alice 购买 3 副 Wayfarer 太阳镜。”
4）**变更——购买多件：**
   “我想买 3 副 Classic 太阳镜和 1 副 Aviator。”


<div style="border:1px solid #93c5fd; border-left:6px solid #3b82f6; background:#eff6ff; border-radius:8px; padding:14px 16px; color:#1e3a8a; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">
  🔎 <strong><code>reseed=True</code> 有什么作用？</strong><br><br>
  调用 <code>customer_service_agent(..., reseed=True)</code> 时，代理会在运行提示前<em>重新初始化</em>演示数据：
  <ul style="margin:8px 0 0 18px;">
    <li><strong>重置</strong> <code>inventory_tbl</code> 为默认产品集合。</li>
    <li><strong>重置</strong> <code>transactions_tbl</code> 为单一的期初余额记录。</li>
    <li>确保一次<strong>干净、可复现</strong>的运行，避免受先前测试影响。</li>
  </ul>
  若希望<strong>保留</strong>当前状态并在既有操作上继续，请设置 <code>reseed=False</code>。
</div>



In [ ]:
prompt = "我想购买 3 副 Classic 太阳镜和 1 副 Aviator 太阳镜。"

out = customer_service_agent(
    prompt,
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="o4-mini",
    temperature=1.0,
    reseed=True,   # 设为 False 以保留当前库存与交易状态
)

## 5. 要点总结

- **让代码成为计划。** 遵循 Andrew 的“以代码为行动”理念，让模型编写串联步骤的 Python（过滤 → 计算 → 更新），然后直接运行。

- **避免脆弱的工具拼凑。** 不再堆砌小工具或 JSON 计划，而是使用 Python/TinyDB——为模型提供一个熟悉且强大的工具箱，用同一个提示处理多种查询形态。

- **保持执行安全且可见。** 在受控命名空间中运行、捕获日志/错误，并查看前/后表——因此你始终清楚变化是什么以及原因。

<div style="border:1px solid #22c55e; border-left:6px solid #16a34a; background:#dcfce7; border-radius:6px; padding:14px 16px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

🎉 <strong>恭喜！</strong>

你已完成本实验并构建了一个<em>具备能动性</em>的客服工作流。你让模型用代码作为计划、安全运行，并用简单的校验保证更新可靠。发生失败时，能给出清晰、可读的原因；成功时，通过前/后快照看到具体变化。

借助该模式——在代码中规划并透明执行——你已准备好设计属于自己的工作流：自动化、安全且易扩展。🚀

</div>
